# Vienna DEM

In [1]:

DEM_10m_raster = r'N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\dem\dem_at_10m\DEM_10m_WIEN_31256.tif'



In [11]:
# should be updated... 
import rasterio


DEM_10m_raster = r'N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\dem\dem_at_10m\DEM_10m_WIEN_31256.tif'
reference_polygon = r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\reference_units\GRID\grid31256\fishnet_100m_31256.shp"


# Open raster file

## done by QGIS:

out_dem_vector =r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\Wien_DEM_by_grid100m.shp"


print ("end")


end


produce a 100m tif from gridded vector dataset:

In [15]:
##

## raster to vector using vector attriubte (working perfectly):

import geopandas as gpd
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_bounds

# Load the polygon layer
polygons = gpd.read_file(r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\Wien_DEM_by_grid100m.shp")

output_raster = r"N:\C2205_FAIRiCUBE\f02_data\d050_data_WIEN\Vienna_gridded_data_collection\100m_GRID\Wien_DEM_by_grid100m_MEAN.tif"


# Define the attribute to use for raster values
attribute_to_rasterize = '_mean'

# Define raster properties
#xmin, ymin, xmax, ymax = polygons.total_bounds

xmin=	-11200
ymin=	331000
xmax=	18800
ymax=	354000

projection ='epsg:31256'


pixel_size = 100  # Adjust as needed


width = int((xmax - xmin) / pixel_size)
height = int((ymax - ymin) / pixel_size)
transform = from_bounds(xmin, ymin, xmax, ymax, width, height)

# Create an empty array to hold rasterized values
raster_array = rasterize(
    [(geom, value) for geom, value in zip(polygons.geometry, polygons[attribute_to_rasterize])],
    out_shape=(height, width),
    transform=transform,
    fill=0,
    all_touched=False,  # or True
    dtype=rasterio.float32
)

# Write the raster to a GeoTIFF file
with rasterio.open(output_raster, "w", driver="GTiff", 
                   height=raster_array.shape[0], 
                   width=raster_array.shape[1], 
                   count=1, 
                   dtype=raster_array.dtype, 
                   crs=polygons.crs, 
                   transform=transform
                   ) as dst:
    dst.write(raster_array, 1)


print ("done")

done


Resulting 100m raster:
![dem](images/example_100m_mean_dem.JPG)




Calculate the dem statistics